### Import libraries

In [29]:
import cv2
import numpy as np
import os

In [30]:
# Skin dataset link: http://cs-chan.com/downloads_skin_dataset.html
data_path = "./Face_Dataset/"
data_path = os.path.expanduser(data_path)
if not os.path.isdir(data_path):
    print("Please download dataset from http://cs-chan.com/downloads_skin_dataset.html and unzip")
else:
    photo_dir = os.path.join(data_path,"Pratheepan_Dataset/FacePhoto")
    mask_dir = os.path.join(data_path,"Ground_Truth/GroundT_FacePhoto")
    photo_list = os.listdir(photo_dir)
    t_photo_dir = os.path.join(data_path,"Pratheepan_Dataset/FamilyPhoto")
    t_mask_dir = os.path.join(data_path,"Ground_Truth/GroundT_FamilyPhoto")
    t_photo_list = os.listdir(t_photo_dir)

In [62]:
def extract_stat(pt_dir,mask_dir):
    """Extract stat from training set for skin color detector
    pt_dir,mask_dir: photo,mask training set directory
    """
    
    h_total = 0
    count = 0
    for id in os.listdir(pt_dir):
        face_image_dir = os.path.join(pt_dir, id)
        mask_image_dir = os.path.join(mask_dir, os.path.splitext(id)[0] + '.png')
        
        face = cv2.imread(face_image_dir)
        mask = cv2.imread(mask_image_dir)
        
        extracted_skin = np.bitwise_and(face, mask)
        
        hsv_face = cv2.cvtColor(face, cv2.COLOR_BGR2HSV)
        
        h_channel = hsv_face[:,:,0]
#         s_channel = hsv_face[:,:,1]
#         v_channel = hsv_face[:,:,2]
        
        mean = np.mean(h_channel)
        std = np.std(h_channel)
        h_total += mean
        
        count += 1
        
#         cv2.imshow('Result', h_channel)
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
    h_mean = h_total / count
    
    return h_total / count

stat = extract_stat(photo_dir,mask_dir)

In [60]:
def infer(img_dir,stat,HSV=True):
    """Inference skin mask from image and stat
    """
    img = cv2.imread(img_dir)
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    h, w, c = img.shape
    mask = np.zeros((h, w))
    for i in range(h):
        for j in range(w):
            if hsv_img[i, j, 0] > stat:
                mask[i, j] = 255
    cv2.imshow('Result', mask)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    
    

In [61]:
mask = cv2.imread(os.path.join(mask_dir,photo_list[1].split('.')[0]+'.png'))[:,:,0]
pred_mask = infer(os.path.join(photo_dir,photo_list[1]),stat,True)

In [20]:
def compute_iou(pred_mask, gt_mask):
    """Computes IoU overlaps between two mask.
    pred_mask, gt_mask: numpy.array([Height, Width])
    """
    
    # flatten masks and compute their areas
    mask1 = np.reshape(pred_mask > .5, (-1, 1)).astype(np.float32)
    mask2 = np.reshape(gt_mask > .5, (-1, 1)).astype(np.float32)
    area1 = np.sum(mask1, axis=0)
    area2 = np.sum(mask2, axis=0)

    # intersections and union
    intersection = np.dot(mask1.T, mask2)
    union = area1[:, None] + area2[None, :] - intersection
    overlaps = intersection / union
    return overlaps[0][0]
  
def compute_overlaps_masks(pred_masks, gt_masks):
    """Computes IoU overlaps between two set of masks.
    pred_masks, gt_masks: lists of prediction mask and groundtruth mask [numpy.array([Height, Width]),...]
    """
    
    iou_score = 0 
    for pred_mask, gt_mask in zip(pred_masks,gt_masks):
      iou_score += compute_iou(pred_mask, gt_mask)
    
    print("Your score is:", iou_score)